# Libraries

In [2]:
# using conda library 'ontogpt_fork'; naming irrelevant as this was previously going to be used to work on a different project.

# increase cell-width
from IPython.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import re
import numpy as np
import os
import glob
from tqdm import tqdm # progress bar tracking

pd.set_option('display.max_rows', None)
pd.set_option('display.width', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

# conll-2003 dataset:
from datasets import load_dataset

import json
import jsonschema
from jsonschema import validate

# prompts for OpenAI

import openai_secret_manager
import openai
import time
import random

import tiktoken
from typing import List, Tuple

from collections import Counter

# Load CoNLL-2003 dataset to compare NER (benchmark)
* CoNLL-2003 is an annotated dataset used to assess NER.

* load_dataset produces datafile split into train, test, validation.
* accessible by subsetting via conll2003["train"], for example, and converting to pandas df.
* furthermore, re ner_tags:
In the Hugging Face datasets library's CoNLL-2003 dataset, the integer values in the "ner_tags" column correspond to the following named entity tags:


    * 0: "O" (Outside) - Indicates that a token does not belong to any named entity.
    * 1: "B-PER" (Beginning of a Person entity) - Marks the beginning of a person's name.
    * 2: "I-PER" (Inside of a Person entity) - Marks the continuation of a person's name.
    * 3: "B-ORG" (Beginning of an Organization entity) - Marks the beginning of an organization's name.
    * 4: "I-ORG" (Inside of an Organization entity) - Marks the continuation of an organization's name.
    * 5: "B-LOC" (Beginning of a Location entity) - Marks the beginning of a location name.
    * 6: "I-LOC" (Inside of a Location entity) - Marks the continuation of a location name.
    * 7: "B-MISC" (Beginning of a Miscellaneous entity) - Marks the beginning of a miscellaneous entity.
    * 8: "I-MISC" (Inside of a Miscellaneous entity) - Marks the continuation of a miscellaneous entity.

In [3]:
conll2003 = load_dataset("conll2003")

Found cached dataset conll2003 (C:/Users/David/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Using only test set.

test_subset = conll2003["test"].select(range(len(conll2003["test"])))
conll2003_test = test_subset.to_pandas()
print(len(conll2003_test))
# 3453 records.

3453


In [5]:
# Rename the 'id' column to 'sent_id'
conll2003_test = conll2003_test.rename(columns={'id': 'sent_id'})

conll2003_test.head()

,sent_id,tokens,pos_tags,chunk_tags,ner_tags
0,0,"[SOCCER, -, JAPAN, GET, LUCKY, WIN, ,, CHINA, ...","[21, 8, 22, 37, 22, 22, 6, 22, 15, 12, 21, 7]","[11, 0, 11, 21, 11, 12, 0, 11, 13, 11, 12, 0]","[0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
1,1,"[Nadim, Ladki]","[22, 22]","[11, 12]","[1, 2]"
2,2,"[AL-AIN, ,, United, Arab, Emirates, 1996-12-06]","[22, 6, 22, 22, 23, 11]","[11, 0, 11, 12, 12, 12]","[5, 0, 5, 6, 6, 0]"
3,3,"[Japan, began, the, defence, of, their, Asian,...","[22, 38, 12, 21, 15, 29, 16, 22, 21, 15, 12, 1...","[11, 21, 11, 12, 13, 11, 12, 12, 12, 13, 11, 1...","[5, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[But, China, saw, their, luck, desert, them, i...","[10, 22, 38, 29, 21, 37, 28, 15, 12, 21, 21, 1...","[0, 11, 21, 11, 12, 21, 11, 13, 11, 12, 12, 13...","[0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


### Re-map list of ints in ner_tags to the actual tags.

In [6]:
# Create string of input data from tokens:

# Convert comma-split list to a single string
conll2003_test['input_text'] = conll2003_test['tokens'].apply(lambda x: ' '.join(x))

In [7]:
conll2003_test.head()

,sent_id,tokens,pos_tags,chunk_tags,ner_tags,input_text
0,0,"[SOCCER, -, JAPAN, GET, LUCKY, WIN, ,, CHINA, ...","[21, 8, 22, 37, 22, 22, 6, 22, 15, 12, 21, 7]","[11, 0, 11, 21, 11, 12, 0, 11, 13, 11, 12, 0]","[0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0]","SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI..."
1,1,"[Nadim, Ladki]","[22, 22]","[11, 12]","[1, 2]",Nadim Ladki
2,2,"[AL-AIN, ,, United, Arab, Emirates, 1996-12-06]","[22, 6, 22, 22, 23, 11]","[11, 0, 11, 12, 12, 12]","[5, 0, 5, 6, 6, 0]","AL-AIN , United Arab Emirates 1996-12-06"
3,3,"[Japan, began, the, defence, of, their, Asian,...","[22, 38, 12, 21, 15, 29, 16, 22, 21, 15, 12, 1...","[11, 21, 11, 12, 13, 11, 12, 12, 12, 13, 11, 1...","[5, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, ...",Japan began the defence of their Asian Cup tit...
4,4,"[But, China, saw, their, luck, desert, them, i...","[10, 22, 38, 29, 21, 37, 28, 15, 12, 21, 21, 1...","[0, 11, 21, 11, 12, 21, 11, 13, 11, 12, 12, 13...","[0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",But China saw their luck desert them in the se...


In [8]:
# Define the mapping dictionary
tag_mapping = {
    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC",
    7: "B-MISC",
    8: "I-MISC"
}

# Map the integers to the corresponding tags
conll2003_test['mapped_ner_tags'] = conll2003_test['ner_tags'].map(lambda tags: [tag_mapping[tag] for tag in tags])

In [9]:
conll2003_test.head()

,sent_id,tokens,pos_tags,chunk_tags,ner_tags,input_text,mapped_ner_tags
0,0,"[SOCCER, -, JAPAN, GET, LUCKY, WIN, ,, CHINA, ...","[21, 8, 22, 37, 22, 22, 6, 22, 15, 12, 21, 7]","[11, 0, 11, 21, 11, 12, 0, 11, 13, 11, 12, 0]","[0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0]","SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","[O, O, B-LOC, O, O, O, O, B-PER, O, O, O, O]"
1,1,"[Nadim, Ladki]","[22, 22]","[11, 12]","[1, 2]",Nadim Ladki,"[B-PER, I-PER]"
2,2,"[AL-AIN, ,, United, Arab, Emirates, 1996-12-06]","[22, 6, 22, 22, 23, 11]","[11, 0, 11, 12, 12, 12]","[5, 0, 5, 6, 6, 0]","AL-AIN , United Arab Emirates 1996-12-06","[B-LOC, O, B-LOC, I-LOC, I-LOC, O]"
3,3,"[Japan, began, the, defence, of, their, Asian,...","[22, 38, 12, 21, 15, 29, 16, 22, 21, 15, 12, 1...","[11, 21, 11, 12, 13, 11, 12, 12, 12, 13, 11, 1...","[5, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, ...",Japan began the defence of their Asian Cup tit...,"[B-LOC, O, O, O, O, O, B-MISC, I-MISC, O, O, O..."
4,4,"[But, China, saw, their, luck, desert, them, i...","[10, 22, 38, 29, 21, 37, 28, 15, 12, 21, 21, 1...","[0, 11, 21, 11, 12, 21, 11, 13, 11, 12, 12, 13...","[0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",But China saw their luck desert them in the se...,"[O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O,..."


In [10]:
# create new col being comma-separated list of any detected orgs.
def extract_organizations(row):
    tokens = row['tokens']
    tags = row['mapped_ner_tags']
    orgs = []
    org = ''
    
    for i, tag in enumerate(tags):
        if tag == 'B-ORG':
            org = tokens[i]
        elif tag == 'I-ORG':
            if org:
                org += ' ' + tokens[i]
        else:
            if org:
                orgs.append(org)
                org = ''
    
    if org:
        orgs.append(org)
    
    return ', '.join(orgs)

# Apply the function to create the 'orgs' column
conll2003_test['orgs'] = conll2003_test.apply(extract_organizations, axis=1)

In [11]:
conll2003_test.head()

,sent_id,tokens,pos_tags,chunk_tags,ner_tags,input_text,mapped_ner_tags,orgs
0,0,"[SOCCER, -, JAPAN, GET, LUCKY, WIN, ,, CHINA, ...","[21, 8, 22, 37, 22, 22, 6, 22, 15, 12, 21, 7]","[11, 0, 11, 21, 11, 12, 0, 11, 13, 11, 12, 0]","[0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0]","SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","[O, O, B-LOC, O, O, O, O, B-PER, O, O, O, O]",
1,1,"[Nadim, Ladki]","[22, 22]","[11, 12]","[1, 2]",Nadim Ladki,"[B-PER, I-PER]",
2,2,"[AL-AIN, ,, United, Arab, Emirates, 1996-12-06]","[22, 6, 22, 22, 23, 11]","[11, 0, 11, 12, 12, 12]","[5, 0, 5, 6, 6, 0]","AL-AIN , United Arab Emirates 1996-12-06","[B-LOC, O, B-LOC, I-LOC, I-LOC, O]",
3,3,"[Japan, began, the, defence, of, their, Asian,...","[22, 38, 12, 21, 15, 29, 16, 22, 21, 15, 12, 1...","[11, 21, 11, 12, 13, 11, 12, 12, 12, 13, 11, 1...","[5, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, ...",Japan began the defence of their Asian Cup tit...,"[B-LOC, O, O, O, O, O, B-MISC, I-MISC, O, O, O...",
4,4,"[But, China, saw, their, luck, desert, them, i...","[10, 22, 38, 29, 21, 37, 28, 15, 12, 21, 21, 1...","[0, 11, 21, 11, 12, 21, 11, 13, 11, 12, 12, 13...","[0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",But China saw their luck desert them in the se...,"[O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O,...",


In [12]:
# Sort the DataFrame in descending order based on the length of the list in 'tokens' column and extract the top 5 rows
top_5_rows = conll2003_test.sort_values(by='tokens', key=lambda x: x.apply(len), ascending=False)[:5]

for i in top_5_rows['tokens']:
    print(i)

['Barbarians' '-' '15' '-' 'Tim' 'Stimpson' '(' 'England' ')' ';' '14' '-'
 'Nigel' 'Walker' '(' 'Wales' ')' ',' '13' '-' 'Allan' 'Bateman' '('
 'Wales' ')' ',' '12' '-' 'Gregor' 'Townsend' '(' 'Scotland' ')' ',' '11'
 '-' 'Tony' 'Underwood' '(' 'England' ')' ';' '10' '-' 'Rob' 'Andrew' '('
 'England' ')' ',' '9' '-' 'Rob' 'Howley' '(' 'Wales' ')' ';' '8' '-'
 'Scott' 'Quinnell' '(' 'Wales' ')' ',' '7' '-' 'Neil' 'Back' '('
 'England' ')' ',' '6' '-' 'Dale' 'McIntosh' '(' 'Pontypridd' ')' ',' '5'
 '-' 'Ian' 'Jones' '(' 'New' 'Zealand' ')' ',' '4' '-' 'Craig' 'Quinnell'
 '(' 'Wales' ')' ',' '3' '-' 'Darren' 'Garforth' '(' 'Leicester' ')' ','
 '2' '-' 'Norm' 'Hewitt' '(' 'New' 'Zealand' ')' ',' '1' '-' 'Nick'
 'Popplewell' '(' 'Ireland' ')' '.']
['Australia' '-' '15' '-' 'Matthew' 'Burke' ';' '14' '-' 'Joe' 'Roff' ','
 '13' '-' 'Daniel' 'Herbert' ',' '12' '-' 'Tim' 'Horan' '(' 'captain' ')'
 ',' '11' '-' 'David' 'Campese' ';' '10' '-' 'Pat' 'Howard' ',' '9' '-'
 'Sam' 'Payne' ';' '8' '-'

In [13]:
conll2003_test_orgs = conll2003_test[conll2003_test['orgs'].str.strip().ne('')]

# Reset the index
conll2003_test_orgs = conll2003_test_orgs.reset_index(drop=True)

print(len(conll2003_test_orgs))
conll2003_test_orgs.head()

# i.e. 1229 rows in test_conll2003 contain an org (at least 1 B-ORG)

1229


,sent_id,tokens,pos_tags,chunk_tags,ner_tags,input_text,mapped_ner_tags,orgs
0,19,"[Japan, ,, co-hosts, of, the, World, Cup, in, ...","[22, 6, 42, 15, 12, 22, 22, 15, 11, 10, 38, 16...","[11, 0, 21, 13, 11, 12, 12, 13, 11, 0, 21, 11,...","[5, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, ...","Japan , co-hosts of the World Cup in 2002 and ...","[B-LOC, O, O, O, O, B-MISC, I-MISC, O, O, O, O...",FIFA
1,22,"[RUGBY, UNION, -, CUTTITTA, BACK, FOR, ITALY, ...","[22, 21, 8, 22, 22, 15, 16, 22, 22, 22, 7]","[11, 12, 0, 11, 12, 13, 11, 12, 12, 12, 0]","[3, 4, 0, 1, 0, 0, 5, 0, 0, 0, 0]",RUGBY UNION - CUTTITTA BACK FOR ITALY AFTER A ...,"[B-ORG, I-ORG, O, B-PER, O, O, B-LOC, O, O, O, O]",RUGBY UNION
2,141,"[Plymouth, 4, Exeter, 1]","[21, 11, 22, 11]","[11, 12, 12, 12]","[3, 0, 3, 0]",Plymouth 4 Exeter 1,"[B-ORG, O, B-ORG, O]","Plymouth, Exeter"
3,144,"[Dutch, forward, Reggie, Blinker, had, his, in...","[16, 16, 22, 22, 38, 29, 16, 21, 40, 15, 22, 1...","[11, 12, 12, 12, 21, 11, 12, 12, 21, 13, 11, 1...","[7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ...",Dutch forward Reggie Blinker had his indefinit...,"[B-MISC, O, B-PER, I-PER, O, O, O, O, O, O, B-...","FIFA, Sheffield Wednesday, Liverpool"
4,145,"[Blinker, missed, his, club, 's, last, two, ga...","[21, 38, 29, 21, 27, 16, 11, 24, 15, 22, 38, 1...","[11, 21, 11, 12, 11, 12, 12, 12, 13, 11, 21, 1...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, ...",Blinker missed his club 's last two games afte...,"[B-PER, O, O, O, O, O, O, O, O, B-ORG, O, O, O...","FIFA, Wednesday, Udinese, Feyenoord"


In [14]:
# word-count
def count_words_without_punctuation(text):
    # Remove punctuation using regex pattern
    text_without_punctuation = re.sub(r'[^\w\s]', '', text)
    # Split text into words and return the count
    return len(text_without_punctuation.split())

conll2003_test_orgs['word_count'] = conll2003_test_orgs['input_text'].apply(count_words_without_punctuation)

largest_word_count = conll2003_test_orgs['word_count'].max()
smallest_word_count = conll2003_test_orgs['word_count'].min()

print(smallest_word_count)
print(largest_word_count)

1
63


In [15]:
# Sort the DataFrame by word count in descending order
df_sorted = conll2003_test_orgs.sort_values(by='word_count', ascending=False)

# Print the head of the sorted DataFrame
df_sorted.head()

,sent_id,tokens,pos_tags,chunk_tags,ner_tags,input_text,mapped_ner_tags,orgs,word_count
42,202,"[Barbarians, -, 15, -, Tim, Stimpson, (, Engla...","[22, 8, 11, 8, 22, 22, 4, 22, 5, 8, 11, 8, 22,...","[11, 0, 11, 12, 12, 12, 0, 11, 0, 0, 11, 12, 1...","[3, 0, 0, 0, 1, 2, 0, 5, 0, 0, 0, 0, 1, 2, 0, ...",Barbarians - 15 - Tim Stimpson ( England ) ; 1...,"[B-ORG, O, O, O, B-PER, I-PER, O, B-LOC, O, O,...",Barbarians,63
622,1841,"[The, nominees, ,, according, to, diplomats, ,...","[12, 24, 6, 39, 35, 24, 6, 41, 8, 22, 22, 15, ...","[11, 12, 0, 13, 13, 11, 0, 21, 0, 11, 12, 13, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 5, 0, 0, ...","The nominees , according to diplomats , are : ...","[O, O, O, O, O, O, O, O, O, B-PER, I-PER, O, B...","U.N., U.N., U.N. General Assembly, Organisatio...",58
486,1312,"[Russian, tax, and, cabinet, authorities, ,, u...","[16, 21, 10, 21, 24, 6, 15, 21, 15, 12, 22, 22...","[11, 12, 12, 12, 12, 0, 13, 11, 13, 11, 12, 12...","[7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, ...","Russian tax and cabinet authorities , under pr...","[B-MISC, O, O, O, O, O, O, O, O, O, B-ORG, I-O...",International Monetary Fund,55
407,1021,"[The, government, also, wants, British, Airway...","[12, 21, 30, 42, 22, 22, 35, 37, 12, 21, 15, 2...","[11, 12, 3, 21, 11, 12, 21, 22, 11, 12, 13, 11...","[0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 3, ...",The government also wants British Airways to d...,"[O, O, O, O, B-ORG, I-ORG, O, O, O, O, O, O, O...","British Airways, USAir, British Airways, American",53
382,950,"[Company, President, Jean, van, Boxmeer, told,...","[22, 22, 22, 22, 22, 38, 22, 15, 12, 21, 15, 2...","[11, 12, 12, 12, 12, 21, 11, 13, 11, 12, 13, 1...","[0, 0, 1, 2, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, ...",Company President Jean van Boxmeer told Reuter...,"[O, O, B-PER, I-PER, I-PER, O, B-ORG, O, O, O,...",Reuters,52


In [13]:
index_sample = 4

# Print data in 'tokens' column at index 4
print(conll2003_test_orgs.at[index_sample, 'tokens'])
print('\n')
# Print data in 'orgs' column at index 4
print(conll2003_test_orgs.at[index_sample, 'mapped_ner_tags']) # 3-4 indicating org
print('\n')
# Print data in 'orgs' column at index 4
print(conll2003_test_orgs.at[index_sample, 'orgs'])

['Blinker' 'missed' 'his' 'club' "'s" 'last' 'two' 'games' 'after' 'FIFA'
 'slapped' 'a' 'worldwide' 'ban' 'on' 'him' 'for' 'appearing' 'to' 'sign'
 'contracts' 'for' 'both' 'Wednesday' 'and' 'Udinese' 'while' 'he' 'was'
 'playing' 'for' 'Feyenoord' '.']


['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O']


FIFA, Wednesday, Udinese, Feyenoord


* Initially thought above was an error (Wednesday) however the full sentence possibly implies that it is actually an org.

In [14]:
# Find the row where 'mapped_ner_tags' contains 'I-ORG'
index_sample = conll2003_test_orgs[conll2003_test_orgs['mapped_ner_tags'
                                                      ].apply(lambda tags: 'I-ORG' in tags)
                                  ].index[1] # change index here for new example.

# Print data in 'tokens' column at the identified index
print(conll2003_test_orgs.at[index_sample, 'tokens'])
print('\n')

# Print data in 'mapped_ner_tags' column at the identified index
print(conll2003_test_orgs.at[index_sample, 'mapped_ner_tags'])
print('\n')

# Print data in 'orgs' column at the identified index
print(conll2003_test_orgs.at[index_sample, 'orgs'])

['Dutch' 'forward' 'Reggie' 'Blinker' 'had' 'his' 'indefinite'
 'suspension' 'lifted' 'by' 'FIFA' 'on' 'Friday' 'and' 'was' 'set' 'to'
 'make' 'his' 'Sheffield' 'Wednesday' 'comeback' 'against' 'Liverpool'
 'on' 'Saturday' '.']


['B-MISC', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'O', 'O', 'O']


FIFA, Sheffield Wednesday, Liverpool


* Above example confirms previous commentary.

# Prompt generation to detect only orgs.
* Attempting to use similar prompt format as used in main KG_construction, for consistency.
* Aim is to benchmark NER.
    * Despite the main research being KGC (triple extraction) -- the triples are centres on the main organisation detected in the Aylien article.
    * Hence it makes sense to benchmark against NER, focussing on orgs.
    * The conll-2003 dataset is also based on news articles.
    
In this iteration, we only work on the conll2003_test_orgs set, which are sentences that always contained at least 1 org (B-ORG tag).

In [15]:
print(len(conll2003_test_orgs))
conll2003_test_orgs.head()

1229


,sent_id,tokens,pos_tags,chunk_tags,ner_tags,input_text,mapped_ner_tags,orgs
0,19,"[Japan, ,, co-hosts, of, the, World, Cup, in, ...","[22, 6, 42, 15, 12, 22, 22, 15, 11, 10, 38, 16...","[11, 0, 21, 13, 11, 12, 12, 13, 11, 0, 21, 11,...","[5, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, ...","Japan , co-hosts of the World Cup in 2002 and ...","[B-LOC, O, O, O, O, B-MISC, I-MISC, O, O, O, O...",FIFA
1,22,"[RUGBY, UNION, -, CUTTITTA, BACK, FOR, ITALY, ...","[22, 21, 8, 22, 22, 15, 16, 22, 22, 22, 7]","[11, 12, 0, 11, 12, 13, 11, 12, 12, 12, 0]","[3, 4, 0, 1, 0, 0, 5, 0, 0, 0, 0]",RUGBY UNION - CUTTITTA BACK FOR ITALY AFTER A ...,"[B-ORG, I-ORG, O, B-PER, O, O, B-LOC, O, O, O, O]",RUGBY UNION
2,141,"[Plymouth, 4, Exeter, 1]","[21, 11, 22, 11]","[11, 12, 12, 12]","[3, 0, 3, 0]",Plymouth 4 Exeter 1,"[B-ORG, O, B-ORG, O]","Plymouth, Exeter"
3,144,"[Dutch, forward, Reggie, Blinker, had, his, in...","[16, 16, 22, 22, 38, 29, 16, 21, 40, 15, 22, 1...","[11, 12, 12, 12, 21, 11, 12, 12, 21, 13, 11, 1...","[7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ...",Dutch forward Reggie Blinker had his indefinit...,"[B-MISC, O, B-PER, I-PER, O, O, O, O, O, O, B-...","FIFA, Sheffield Wednesday, Liverpool"
4,145,"[Blinker, missed, his, club, 's, last, two, ga...","[21, 38, 29, 21, 27, 16, 11, 24, 15, 22, 38, 1...","[11, 21, 11, 12, 11, 12, 12, 12, 13, 11, 21, 1...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, ...",Blinker missed his club 's last two games afte...,"[B-PER, O, O, O, O, O, O, O, O, B-ORG, O, O, O...","FIFA, Wednesday, Udinese, Feyenoord"


In [16]:
# Create a function to generate the prompt based on the row values
def generate_prompt(row):

    input_text = row['input_text']
    
    prompt = f'''I need you to show me organisations in the provided sentence. Do not show me any other entity types.
    ###
    Desired response format: ###
    org1, org2, org3
    ###
    Desired response if no organisations found: ###
    no_orgs
    ###
    For example, if I gave you: ###
    "James, living in Japan, missed his club 's last two games after FIFA slapped a worldwide ban on him for appearing to sign contracts for both Wednesday and Udinese while he was playing for Feyenoord .",
    you would only give me:
    FIFA, Wednesday, Udinese, Feyenoord
    ###
    The provided sentence is: ###
    {input_text}
    ###
    '''
    
    return prompt

# Add a new 'prompt' column by applying the generate_prompt function to each row
conll2003_test_orgs['prompt'] = conll2003_test_orgs.apply(generate_prompt, axis=1)
conll2003_test_orgs['prompt'] = conll2003_test_orgs['prompt'].astype(str)


In [17]:
print(len(conll2003_test_orgs))
conll2003_test_orgs.head()

1229


,sent_id,tokens,pos_tags,chunk_tags,ner_tags,input_text,mapped_ner_tags,orgs,prompt
0,19,"[Japan, ,, co-hosts, of, the, World, Cup, in, ...","[22, 6, 42, 15, 12, 22, 22, 15, 11, 10, 38, 16...","[11, 0, 21, 13, 11, 12, 12, 13, 11, 0, 21, 11,...","[5, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, ...","Japan , co-hosts of the World Cup in 2002 and ...","[B-LOC, O, O, O, O, B-MISC, I-MISC, O, O, O, O...",FIFA,I need you to show me organisations in the pro...
1,22,"[RUGBY, UNION, -, CUTTITTA, BACK, FOR, ITALY, ...","[22, 21, 8, 22, 22, 15, 16, 22, 22, 22, 7]","[11, 12, 0, 11, 12, 13, 11, 12, 12, 12, 0]","[3, 4, 0, 1, 0, 0, 5, 0, 0, 0, 0]",RUGBY UNION - CUTTITTA BACK FOR ITALY AFTER A ...,"[B-ORG, I-ORG, O, B-PER, O, O, B-LOC, O, O, O, O]",RUGBY UNION,I need you to show me organisations in the pro...
2,141,"[Plymouth, 4, Exeter, 1]","[21, 11, 22, 11]","[11, 12, 12, 12]","[3, 0, 3, 0]",Plymouth 4 Exeter 1,"[B-ORG, O, B-ORG, O]","Plymouth, Exeter",I need you to show me organisations in the pro...
3,144,"[Dutch, forward, Reggie, Blinker, had, his, in...","[16, 16, 22, 22, 38, 29, 16, 21, 40, 15, 22, 1...","[11, 12, 12, 12, 21, 11, 12, 12, 21, 13, 11, 1...","[7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ...",Dutch forward Reggie Blinker had his indefinit...,"[B-MISC, O, B-PER, I-PER, O, O, O, O, O, O, B-...","FIFA, Sheffield Wednesday, Liverpool",I need you to show me organisations in the pro...
4,145,"[Blinker, missed, his, club, 's, last, two, ga...","[21, 38, 29, 21, 27, 16, 11, 24, 15, 22, 38, 1...","[11, 21, 11, 12, 11, 12, 12, 12, 13, 11, 21, 1...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, ...",Blinker missed his club 's last two games afte...,"[B-PER, O, O, O, O, O, O, O, O, B-ORG, O, O, O...","FIFA, Wednesday, Udinese, Feyenoord",I need you to show me organisations in the pro...


In [18]:
# export the input data with prompts:

conll2003_test_orgs.to_excel('./benchmarking/input/conll2003_test_orgs.xlsx')

In [19]:
# convert to list to explore prompts:

all_prompts = conll2003_test_orgs['prompt'].tolist()
all_prompts[0]

'I need you to show me organisations in the provided sentence. Do not show me any other entity types.\n    ###\n    Desired response format: ###\n    org1, org2, org3\n    ###\n    Desired response if no organisations found: ###\n    no_orgs\n    ###\n    For example, if I gave you: ###\n    "James, living in Japan, missed his club \'s last two games after FIFA slapped a worldwide ban on him for appearing to sign contracts for both Wednesday and Udinese while he was playing for Feyenoord .",\n    you would only give me:\n    FIFA, Wednesday, Udinese, Feyenoord\n    ###\n    The provided sentence is: ###\n    Japan , co-hosts of the World Cup in 2002 and ranked 20th in the world by FIFA , are favourites to regain their title here .\n    ###\n    '

# Exploring prompts

## Investigate average length of tokens

* Important for pricing forecast and LLM API restrictions

### with tiktoken

In [20]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
#encoding = tiktoken.get_encoding("cl100k_base")

In [21]:
# fns to count strings in list of prompts:

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def count_tokens_in_list(prompt_list: list, encoding_name: str) -> list:
    """Returns a list of integers representing the number of tokens in each string in the input list."""
    token_counts = []
    for prompt in prompt_list:
        num_tokens = num_tokens_from_string(prompt, encoding_name)
        token_counts.append(num_tokens)
    return token_counts


In [22]:
encoding_name = "cl100k_base" # used for gpt-3.5-turbo
token_counts = count_tokens_in_list(all_prompts, encoding_name)

In [23]:
min_tokens = min(token_counts)
max_tokens = max(token_counts)
total_tokens = sum(i for i in token_counts if isinstance(i, int))
average_tokens = total_tokens / len(all_prompts)

print(f"Number of tokens on the smallest prompt: {min_tokens}")
print(f"Number of tokens on the largest prompt: {max_tokens}")
print(f"Total number of tokens for all prompts: {total_tokens}")
print(f"Average number of tokens in all_prompts: {average_tokens}")

Number of tokens on the smallest prompt: 147
Number of tokens on the largest prompt: 300
Total number of tokens for all prompts: 204113
Average number of tokens in all_prompts: 166.08055329536208


In [24]:
# Estimating max response tokens if prompt works correctly (update to reflect prompt used)

#num_tokens_from_string("facing risk: yes. type of risk: thisis some text for a risk type.", "cl100k_base")

num_tokens_from_string(
    
    '''
    [org1, org2, org3, 3oigh3io4gh34, 3o4igh34iogh34, 3o4igh34iogh
    '''
                       ,"cl100k_base" )

42

### Truncate tokens of long prompts

**** NO NEED TO TRUNCATE PROMPTS WORKING WITH conll2003_test_orgs ****
**** All are below token input threshold ****

* gpt-3.5-turbo has max tokens of 4,096 tokens
* This includes prompt and response tokens combined.
* response tokens should be short due to the attempt at prompt restrictions;
    * i.e. Provide answers only in the format of <facing risk: <'yes'/'no'>. type of risk: < risk type >.> and nothing else.
* so a generous estimate of response tokens would be 100, providing gpt-3.5-turbo successfully adheres to above prompting.
* Therefore truncate prompt tokens to 3500 to be safe.

# Feed prompts into LLM

* Initially was working from a list of prompts.
* Modified to work from df and append response directly to df in new 'responses' col.
    * Allows for linking to sent_id without feeding id into prompt.

In [25]:
# Create new df to contain responses at corresponding rows.

# MODIFY TO SPECIFY AMOUNT OF INPUT.

conll2003_test_orgs_responses = conll2003_test_orgs.copy()
conll2003_test_orgs_responses.head()

,sent_id,tokens,pos_tags,chunk_tags,ner_tags,input_text,mapped_ner_tags,orgs,prompt
0,19,"[Japan, ,, co-hosts, of, the, World, Cup, in, ...","[22, 6, 42, 15, 12, 22, 22, 15, 11, 10, 38, 16...","[11, 0, 21, 13, 11, 12, 12, 13, 11, 0, 21, 11,...","[5, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, ...","Japan , co-hosts of the World Cup in 2002 and ...","[B-LOC, O, O, O, O, B-MISC, I-MISC, O, O, O, O...",FIFA,I need you to show me organisations in the pro...
1,22,"[RUGBY, UNION, -, CUTTITTA, BACK, FOR, ITALY, ...","[22, 21, 8, 22, 22, 15, 16, 22, 22, 22, 7]","[11, 12, 0, 11, 12, 13, 11, 12, 12, 12, 0]","[3, 4, 0, 1, 0, 0, 5, 0, 0, 0, 0]",RUGBY UNION - CUTTITTA BACK FOR ITALY AFTER A ...,"[B-ORG, I-ORG, O, B-PER, O, O, B-LOC, O, O, O, O]",RUGBY UNION,I need you to show me organisations in the pro...
2,141,"[Plymouth, 4, Exeter, 1]","[21, 11, 22, 11]","[11, 12, 12, 12]","[3, 0, 3, 0]",Plymouth 4 Exeter 1,"[B-ORG, O, B-ORG, O]","Plymouth, Exeter",I need you to show me organisations in the pro...
3,144,"[Dutch, forward, Reggie, Blinker, had, his, in...","[16, 16, 22, 22, 38, 29, 16, 21, 40, 15, 22, 1...","[11, 12, 12, 12, 21, 11, 12, 12, 21, 13, 11, 1...","[7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ...",Dutch forward Reggie Blinker had his indefinit...,"[B-MISC, O, B-PER, I-PER, O, O, O, O, O, O, B-...","FIFA, Sheffield Wednesday, Liverpool",I need you to show me organisations in the pro...
4,145,"[Blinker, missed, his, club, 's, last, two, ga...","[21, 38, 29, 21, 27, 16, 11, 24, 15, 22, 38, 1...","[11, 21, 11, 12, 11, 12, 12, 12, 13, 11, 21, 1...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, ...",Blinker missed his club 's last two games afte...,"[B-PER, O, O, O, O, O, O, O, O, B-ORG, O, O, O...","FIFA, Wednesday, Udinese, Feyenoord",I need you to show me organisations in the pro...


In [26]:
openai.api_key = os.getenv('OPENAI_KEY')

In [27]:
# model:
GPT_MODEL = "gpt-3.5-turbo"

# Retry parameters
max_retries = 3
retry_delay = 5  # in seconds

# Initialize a new column 'responses' in the dataframe
conll2003_test_orgs_responses.loc[:, 'responses'] = ''

def generate_responses(input_df):
    for idx, row in tqdm(input_df.iterrows(), total=len(input_df), desc="Generating responses"):
        try:
            query = row['prompt']

            if query is None or query.strip() == '':
                print(f"Empty prompt at index {idx}. Skipping this row.")
                continue

            retries = 0
            while retries < max_retries:
                try:
                    response = openai.ChatCompletion.create(
                        model=GPT_MODEL,
                        messages=[
                            {"role": "system", "content": "You will detect organisations in the provided sentence."},
                            {"role": "user", "content": query}, # In general, gpt-3.5-turbo-0301 does not pay strong attention to the system message, and therefore important instructions are often better placed in a user message.
                        ],
                        temperature=0,
                        max_tokens=600,  # max tokens in response. token limit (4096) must be < largest query + message content + max_tokens
                    )

                    response_content = response['choices'][0]['message']['content']
                    print(response_content)  # Print just the message content

                    # Add the response to the 'responses' column of the dataframe
                    input_df.loc[idx, 'responses'] = response_content

                    break  # Break out of the retry loop if successful

                except Exception as e:
                    print(f"Error generating response for prompt at index {idx}: {str(e)}")
                    print(f"Query: {query}")
                    retries += 1
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)

            if retries == max_retries:
                print(f"Max retries reached for prompt at index {idx}. Skipping this row.")

            # Add a delay between requests to avoid overwhelming the API
            time.sleep(random.uniform(0.5, 1.5))

        except Exception as e:
            print(f"Error generating response for prompt at index {idx}: {str(e)}")
            print(f"Query: {query}")

In [28]:
generate_responses(conll2003_test_orgs_responses)

Generating responses:   0%|          | 0/1229 [00:00<?, ?it/s]

Japan, World Cup, FIFA


Generating responses:   0%|          | 1/1229 [00:03<1:17:20,  3.78s/it]

ITALY


Generating responses:   0%|          | 2/1229 [00:05<53:18,  2.61s/it]  

...

Generating responses: 100%|█████████▉| 1227/1229 [1:06:06<00:03,  1.75s/it]

no_orgs


Generating responses: 100%|█████████▉| 1228/1229 [1:06:08<00:01,  1.63s/it]

Leeds United, England, World Cup


Generating responses: 100%|██████████| 1229/1229 [1:06:10<00:00,  3.23s/it]


In [29]:
conll2003_test_orgs_responses.head()

,sent_id,tokens,pos_tags,chunk_tags,ner_tags,input_text,mapped_ner_tags,orgs,prompt,responses
0,19,"[Japan, ,, co-hosts, of, the, World, Cup, in, ...","[22, 6, 42, 15, 12, 22, 22, 15, 11, 10, 38, 16...","[11, 0, 21, 13, 11, 12, 12, 13, 11, 0, 21, 11,...","[5, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, ...","Japan , co-hosts of the World Cup in 2002 and ...","[B-LOC, O, O, O, O, B-MISC, I-MISC, O, O, O, O...",FIFA,I need you to show me organisations in the pro...,"Japan, World Cup, FIFA"
1,22,"[RUGBY, UNION, -, CUTTITTA, BACK, FOR, ITALY, ...","[22, 21, 8, 22, 22, 15, 16, 22, 22, 22, 7]","[11, 12, 0, 11, 12, 13, 11, 12, 12, 12, 0]","[3, 4, 0, 1, 0, 0, 5, 0, 0, 0, 0]",RUGBY UNION - CUTTITTA BACK FOR ITALY AFTER A ...,"[B-ORG, I-ORG, O, B-PER, O, O, B-LOC, O, O, O, O]",RUGBY UNION,I need you to show me organisations in the pro...,ITALY
2,141,"[Plymouth, 4, Exeter, 1]","[21, 11, 22, 11]","[11, 12, 12, 12]","[3, 0, 3, 0]",Plymouth 4 Exeter 1,"[B-ORG, O, B-ORG, O]","Plymouth, Exeter",I need you to show me organisations in the pro...,no_orgs
3,144,"[Dutch, forward, Reggie, Blinker, had, his, in...","[16, 16, 22, 22, 38, 29, 16, 21, 40, 15, 22, 1...","[11, 12, 12, 12, 21, 11, 12, 12, 21, 13, 11, 1...","[7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ...",Dutch forward Reggie Blinker had his indefinit...,"[B-MISC, O, B-PER, I-PER, O, O, O, O, O, O, B-...","FIFA, Sheffield Wednesday, Liverpool",I need you to show me organisations in the pro...,"FIFA, Sheffield Wednesday, Liverpool"
4,145,"[Blinker, missed, his, club, 's, last, two, ga...","[21, 38, 29, 21, 27, 16, 11, 24, 15, 22, 38, 1...","[11, 21, 11, 12, 11, 12, 12, 12, 13, 11, 21, 1...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, ...",Blinker missed his club 's last two games afte...,"[B-PER, O, O, O, O, O, O, O, O, B-ORG, O, O, O...","FIFA, Wednesday, Udinese, Feyenoord",I need you to show me organisations in the pro...,"FIFA, Wednesday, Udinese, Feyenoord"


### Minor post-processing of responses

In [30]:
conll2003_test_orgs_responses.loc[conll2003_test_orgs_responses['responses'] == 'no_orgs', 'responses'] = ""

## Identify hard matches between 'orgs' and 'responses_orgs_only'.
* orgs being the organisations detected in the conll2003_test dataset.
* responses_orgs_only being the orgs detected by LLM.

In [31]:
# Sorting the items in the 'orgs' column
conll2003_test_orgs_responses['orgs'] = conll2003_test_orgs_responses['orgs'].apply(lambda x: ','.join(sorted(x.split(','))) if pd.notnull(x) else x)

# Sorting the items in the 'responses' column
conll2003_test_orgs_responses['responses'] = conll2003_test_orgs_responses['responses'].apply(lambda x: ','.join(sorted(x.split(','))) if pd.notnull(x) else x)

# Computing the 'hard_match' column
conll2003_test_orgs_responses['hard_match'] = conll2003_test_orgs_responses.apply(lambda row: np.all(row['orgs'] == row['responses']), axis=1)

# Counting the True values in the 'hard_match' column
true_count = conll2003_test_orgs_responses['hard_match'].sum()
print("Count where 'hard_match' is True:", true_count)

Count where 'hard_match' is True: 507


## Identify partial matches
* Defined where at least one, but not all, items in the 'orgs' col (list of orgs) is present in the 'responses_orgs_only' col

In [32]:
conll2003_test_orgs_responses['partial_match'] = False  # Initialize the column with False values

# Iterate over each row using .iterrows()
for idx, row in conll2003_test_orgs_responses.iterrows():
    orgs = row['orgs']
    responses_orgs_only = row['responses']
    partial_match = (
        (any(org in responses_orgs_only.split(', ') for org in orgs) and
         not all(org in responses_orgs_only.split(', ') for org in orgs))
    ) or (
        any(org in orgs.split(', ') for org in responses_orgs_only.split(', ')) and
        not all(org in orgs.split(', ') for org in responses_orgs_only.split(', '))
    )
    conll2003_test_orgs_responses.loc[idx, 'partial_match'] = partial_match

# Calculate the sum where 'partial_match' is True
soft_true_count = conll2003_test_orgs_responses['partial_match'].sum()
print("Sum where 'partial_match' is True:", soft_true_count)

# Display the updated DataFrame
conll2003_test_orgs_responses.head()


Sum where 'partial_match' is True: 19


,sent_id,tokens,pos_tags,chunk_tags,ner_tags,input_text,mapped_ner_tags,orgs,prompt,responses,hard_match,partial_match
0,19,"[Japan, ,, co-hosts, of, the, World, Cup, in, ...","[22, 6, 42, 15, 12, 22, 22, 15, 11, 10, 38, 16...","[11, 0, 21, 13, 11, 12, 12, 13, 11, 0, 21, 11,...","[5, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, ...","Japan , co-hosts of the World Cup in 2002 and ...","[B-LOC, O, O, O, O, B-MISC, I-MISC, O, O, O, O...",FIFA,I need you to show me organisations in the pro...,"FIFA, World Cup,Japan",False,False
1,22,"[RUGBY, UNION, -, CUTTITTA, BACK, FOR, ITALY, ...","[22, 21, 8, 22, 22, 15, 16, 22, 22, 22, 7]","[11, 12, 0, 11, 12, 13, 11, 12, 12, 12, 0]","[3, 4, 0, 1, 0, 0, 5, 0, 0, 0, 0]",RUGBY UNION - CUTTITTA BACK FOR ITALY AFTER A ...,"[B-ORG, I-ORG, O, B-PER, O, O, B-LOC, O, O, O, O]",RUGBY UNION,I need you to show me organisations in the pro...,ITALY,False,False
2,141,"[Plymouth, 4, Exeter, 1]","[21, 11, 22, 11]","[11, 12, 12, 12]","[3, 0, 3, 0]",Plymouth 4 Exeter 1,"[B-ORG, O, B-ORG, O]","Exeter,Plymouth",I need you to show me organisations in the pro...,,False,False
3,144,"[Dutch, forward, Reggie, Blinker, had, his, in...","[16, 16, 22, 22, 38, 29, 16, 21, 40, 15, 22, 1...","[11, 12, 12, 12, 21, 11, 12, 12, 21, 13, 11, 1...","[7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ...",Dutch forward Reggie Blinker had his indefinit...,"[B-MISC, O, B-PER, I-PER, O, O, O, O, O, O, B-...","Liverpool, Sheffield Wednesday,FIFA",I need you to show me organisations in the pro...,"Liverpool, Sheffield Wednesday,FIFA",True,False
4,145,"[Blinker, missed, his, club, 's, last, two, ga...","[21, 38, 29, 21, 27, 16, 11, 24, 15, 22, 38, 1...","[11, 21, 11, 12, 11, 12, 12, 12, 13, 11, 21, 1...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, ...",Blinker missed his club 's last two games afte...,"[B-PER, O, O, O, O, O, O, O, O, B-ORG, O, O, O...","Feyenoord, Udinese, Wednesday,FIFA",I need you to show me organisations in the pro...,"Feyenoord, Udinese, Wednesday,FIFA",True,False


## Identify additional orgs found
* Being where responses_orgs_only contains an item (word in comma-separated list) that is not in the orgs col (which is also a comma-separated list of orgs)
* 2 new cols: additional_orgs_found, with a list of any new orgs found.
* count_new_orgs_found, being sum of any new orgs in each row.

In [33]:
conll2003_test_orgs_responses['additional_orgs_found'] = conll2003_test_orgs_responses.apply(lambda row: [org for org in row['responses'].split(', ') if org not in row['orgs']], axis=1)

conll2003_test_orgs_responses.loc[:, 'count_new_orgs_found'] = conll2003_test_orgs_responses['additional_orgs_found'].apply(lambda x: len(x))

sum_count_new_orgs_found = conll2003_test_orgs_responses['count_new_orgs_found'].sum()
print("Sum of 'count_new_orgs_found':", sum_count_new_orgs_found)

# Display the updated DataFrame
conll2003_test_orgs_responses.head()


Sum of 'count_new_orgs_found': 359


,sent_id,tokens,pos_tags,chunk_tags,ner_tags,input_text,mapped_ner_tags,orgs,prompt,responses,hard_match,partial_match,additional_orgs_found,count_new_orgs_found
0,19,"[Japan, ,, co-hosts, of, the, World, Cup, in, ...","[22, 6, 42, 15, 12, 22, 22, 15, 11, 10, 38, 16...","[11, 0, 21, 13, 11, 12, 12, 13, 11, 0, 21, 11,...","[5, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, ...","Japan , co-hosts of the World Cup in 2002 and ...","[B-LOC, O, O, O, O, B-MISC, I-MISC, O, O, O, O...",FIFA,I need you to show me organisations in the pro...,"FIFA, World Cup,Japan",False,False,"[ FIFA, World Cup,Japan]",2
1,22,"[RUGBY, UNION, -, CUTTITTA, BACK, FOR, ITALY, ...","[22, 21, 8, 22, 22, 15, 16, 22, 22, 22, 7]","[11, 12, 0, 11, 12, 13, 11, 12, 12, 12, 0]","[3, 4, 0, 1, 0, 0, 5, 0, 0, 0, 0]",RUGBY UNION - CUTTITTA BACK FOR ITALY AFTER A ...,"[B-ORG, I-ORG, O, B-PER, O, O, B-LOC, O, O, O, O]",RUGBY UNION,I need you to show me organisations in the pro...,ITALY,False,False,[ITALY],1
2,141,"[Plymouth, 4, Exeter, 1]","[21, 11, 22, 11]","[11, 12, 12, 12]","[3, 0, 3, 0]",Plymouth 4 Exeter 1,"[B-ORG, O, B-ORG, O]","Exeter,Plymouth",I need you to show me organisations in the pro...,,False,False,[],0
3,144,"[Dutch, forward, Reggie, Blinker, had, his, in...","[16, 16, 22, 22, 38, 29, 16, 21, 40, 15, 22, 1...","[11, 12, 12, 12, 21, 11, 12, 12, 21, 13, 11, 1...","[7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ...",Dutch forward Reggie Blinker had his indefinit...,"[B-MISC, O, B-PER, I-PER, O, O, O, O, O, O, B-...","Liverpool, Sheffield Wednesday,FIFA",I need you to show me organisations in the pro...,"Liverpool, Sheffield Wednesday,FIFA",True,False,[],0
4,145,"[Blinker, missed, his, club, 's, last, two, ga...","[21, 38, 29, 21, 27, 16, 11, 24, 15, 22, 38, 1...","[11, 21, 11, 12, 11, 12, 12, 12, 13, 11, 21, 1...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, ...",Blinker missed his club 's last two games afte...,"[B-PER, O, O, O, O, O, O, O, O, B-ORG, O, O, O...","Feyenoord, Udinese, Wednesday,FIFA",I need you to show me organisations in the pro...,"Feyenoord, Udinese, Wednesday,FIFA",True,False,[],0


# Final export for manual verification

In [34]:
# export to excel for manual verification:
conll2003_test_orgs_responses.to_excel("./benchmarking/results/conll2003_test_orgs_RESPONSES.xlsx")

# export to pickle in case of future use.
conll2003_test_orgs_responses.to_pickle("./benchmarking/results/conll2003_test_orgs_RESPONSES.pkl")